In [1]:
from azureml.core import Workspace
from azureml.core.authentication import MsiAuthentication
msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="7704663e-8944-4300-b951-6462d41ab638", # put in .json
resource_group="arg-rt-pacedev-SpaceTech", # also in .json
workspace_name="aml-WS-spacetech001", # also
auth=msi_auth
)

keyvault = ws.get_default_keyvault()
conn_str = keyvault.get_secret("asartpacedevstvdsm001-connstr")

# Baseline Dataset

In [18]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

veg_type = 'pv'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

baseline_df = pd.DataFrame()

blob_names = container_client.list_blobs(name_starts_with=f'training/traininginput_{veg_type}_v2')

for blob_name in blob_names:
    blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
    df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
    baseline_df = baseline_df.append(df)

baseline_df.drop(columns=['actual_3m', 'actual_6m', 'actual_9m', 'actual_12m', 'lat_4326', 'lon_4326'], inplace=True)

output = baseline_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/baseline/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9EB5923BBE8BD"',
 'last_modified': datetime.datetime(2022, 2, 8, 23, 17, 13, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'y\xb6\x16\x1cty\xae\xc6'),
 'client_request_id': '3f8423d0-8935-11ec-8b2c-1bbaf30259bc',
 'request_id': 'd86ff50d-401e-006f-6c42-1da8b3000000',
 'version': '2020-06-12',
 'version_id': '2022-02-08T23:17:13.3789914Z',
 'date': datetime.datetime(2022, 2, 8, 23, 17, 12, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

# Pipeline for monthly target dataset

In [5]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

end_month = '2022-01'
veg_type = 'bs'

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

target_df = pd.DataFrame()

for month in months:

    blob_names = container_client.list_blobs(name_starts_with=f'{month[:4]}/{month[5:]}/fused/{veg_type}_v1')

    for blob_name in blob_names:
        blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
        df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
        if datetime.date(int(end_month[:4]), int(end_month[5:]), 1) >= datetime.date(2021, 12, 1):
            try:
                df = df.sample(n=100)
            except:
                pass
        target_df = target_df.append(df)

target_df.drop(columns=['filename', 'lat', 'lon'], inplace=True)
target_df.insert(1, 'month', target_df.iloc[:, 1:13].values.argmax(axis=1) + 1)
target_df.drop(columns=['month1','month2','month3','month4','month5','month6','month7','month8','month9','month10','month11','month12'], inplace=True)

output = target_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/target/{end_month}/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9F2A3D871514A"',
 'last_modified': datetime.datetime(2022, 2, 18, 5, 59, 37, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\r\x98\xf5\x9a\x8c\x91\xa1\xab'),
 'client_request_id': 'f43bcf16-907f-11ec-a2ac-7ba6400d1700',
 'request_id': '43b88ccf-201e-0030-538c-245a4d000000',
 'version': '2020-06-12',
 'version_id': '2022-02-18T05:59:37.4531914Z',
 'date': datetime.datetime(2022, 2, 18, 5, 59, 36, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [3]:
target_df

year  month       evi  evi_prev       u10  u10_prev       v10  v10_prev  \
0   2021.0      2  0.007483  0.173067 -0.243666 -0.373275 -0.613297  0.569317   
1   2021.0      2  0.148059  0.235237 -0.243691 -0.373272 -0.613412  0.569273   
2   2021.0      2 -0.045407  0.130346 -0.244096 -0.373387 -0.614622  0.568762   
3   2021.0      2  0.061590  0.191242 -0.244160 -0.373458 -0.614598  0.568747   
4   2021.0      2 -0.036851  0.139486 -0.244121 -0.373384 -0.614736  0.568719   
..     ...    ...       ...       ...       ...       ...       ...       ...   
44  2022.0      1  0.197016  0.180484  0.172054  0.172054  0.911197  0.911197   
0   2022.0      1  0.205232  0.173347  0.153107  0.153107  0.894372  0.894372   
1   2022.0      1  0.205559  0.171916  0.153022  0.153022  0.894338  0.894338   
2   2022.0      1  0.256164  0.233440  0.153135  0.153135  0.894488  0.894488   
3   2022.0      1  0.258836  0.221952  0.153049  0.153049  0.894455  0.894455   

           t2m    t2m_prev  ...  tp_prev2  tp_prev3     swvl1  swvl1_prev  \
0   299.519380  304.672994  ...  0.002968  0.000266  0.321260    0.142561   
1   299.519251  304.672794  ...  0.002967  0.000266  0.321164    0.142442   
2   299.517803  304.670782  ...  0.002965  0.000265  0.320036    0.141034   
3   299.517787  304.670876  ...  0.002965  0.000265  0.320003    0.140978   
4   299.517674  304.670582  ...  0.002965  0.000265  0.319940    0.140916   
..         ...         ...  ...       ...       ...       ...         ...   
44  307.795053  307.795053  ...  0.000731  0.000001  0.067081    0.067081   
0   307.882885  307.882885  ...  0.000732  0.000001  0.072690    0.072690   
1   307.883291  307.883291  ...  0.000732  0.000001  0.072727    0.072727   
2   307.882777  307.882777  ...  0.000732  0.000001  0.072716    0.072716   
3   307.883183  307.883183  ...  0.000732  0.000001  0.072754    0.072754   

       co2  co2_prev  season_ch_3mon  season_ch_6mon  season_ch_9mon  \
0   415.28    414.77        0.094858       -0.063568       -0.067084   
1   415.28    414.77        0.096976       -0.107570       -0.117953   
2   415.28    414.77       -0.030692       -0.059880       -0.031070   
3   415.28    414.77       -0.053329       -0.084913       -0.061580   
4   415.28    414.77       -0.023950       -0.053391       -0.036470   
..     ...       ...             ...             ...             ...   
44  415.68    415.68       -0.001338       -0.186872       -0.198951   
0   415.68    415.68        0.038138       -0.055770       -0.104642   
1   415.68    415.68        0.015821       -0.046683       -0.106280   
2   415.68    415.68        0.029221       -0.116887       -0.218772   
3   415.68    415.68        0.029012       -0.108677       -0.248693   

    season_ch_12mon  
0         -0.203120  
1         -0.173682  
2         -0.201158  
3         -0.146663  
4         -0.191294  
..              ...  
44        -0.172123  
0         -0.085781  
1         -0.076417  
2         -0.209513  
3         -0.223561  

[244704 rows x 32 columns]

In [17]:
output = target_df.to_csv(index=False, encoding='utf-8', header=True)

blob_client = get_blob_client(filename=f'data_drift/target/{end_month}/{veg_type}.csv', conn_str=conn_str)
blob_client.upload_blob(output, overwrite=True, blob_type='BlockBlob')

{'etag': '"0x8D9EB57A7C0D902"',
 'last_modified': datetime.datetime(2022, 2, 8, 23, 6, 35, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\r\x98\xf5\x9a\x8c\x91\xa1\xab'),
 'client_request_id': 'c38ba678-8933-11ec-8b2c-1bbaf30259bc',
 'request_id': 'a105bcb5-e01e-000f-0240-1ded91000000',
 'version': '2020-06-12',
 'version_id': '2022-02-08T23:06:35.8771480Z',
 'date': datetime.datetime(2022, 2, 8, 23, 6, 35, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}